In [1]:
from sklearn.cluster import KMeans
from skimage.io import imread
import pylab
import skimage
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
#приведение всех значений в интервал от 0 до 1
image = skimage.img_as_float(imread('parrots.jpg'))
x,y,d = image.shape
#приведение к матрицы объекты-признаки
img_array = pd.DataFrame(np.reshape(image, (x*y, d)), columns=['R', 'G', 'B'])

In [3]:
#кластеризация по n_clusters кластерам
#и вовзращение двух измененных картинок по указанным параметрам
def Clustering(image_array, n_clusters=8):
    #копия обязательно иначе накапливается ошибка и результат получается неверный
    #ввиду того, что Python работает с объектами, как с ссылками
    img_array = image_array.copy()
    print ('Count of clusters: ' + str(n_clusters))
    kmeans = KMeans(n_clusters=n_clusters,init='k-means++', random_state=241).fit(img_array)
    img_array['cluster'] = kmeans.labels_
    #в инете почему-то пишут как ниже, резульат идентичный плучается в итоге все равно
    #img_array['cluster'] = kmeans.predict(img_array)
    
    #получаем картинку со средними значениями RGB по кластерам
    means = img_array.groupby('cluster').mean().values
    X_mean = [means[c] for c in img_array['cluster'].values]
    mean_image = np.reshape(X_mean, (x, y, d))
    
    #получаем картинку со медианными значениями RGB по кластерам
    medians = img_array.groupby('cluster').median().values
    X_median = [medians[c] for c in img_array['cluster'].values]
    median_image = np.reshape(X_median, (x, y, d))
    return mean_image, median_image

#подсчет метрики Peak signal-to-noise ratio
def PSNR(img, img_pred):
    MSE = np.mean((image - img_pred) ** 2)
    return 20*np.log10(1.0)-10*np.log10(MSE)

In [4]:
for i in range(1, 21):
    mean_img, median_img = Clustering(img_array, i)
    psnr_mean, psnr_median = PSNR(image, mean_img), PSNR(image, median_img)
    print(psnr_mean, psnr_median)
    #skimage.io.imshow(mean_image) #для показа получившегося изображения
    #доведение до указанной точности в условии задачи
    if (psnr_mean > 20 or psnr_median > 20):
        break

Count of clusters: 1
9.818835544771293 9.427840460507396
Count of clusters: 2
12.080177084931087 11.655077266907849
Count of clusters: 3
13.153487044607246 12.774948175099809
Count of clusters: 4
14.3678179408688 14.005928721924676
Count of clusters: 5
15.527146197305989 15.17790907734613
Count of clusters: 6
16.543994096630186 16.051176778469276
Count of clusters: 7
17.64099250863652 17.339368491425024
Count of clusters: 8
18.436829014063576 18.1379714433961
Count of clusters: 9
19.112672735788138 18.81388735617062
Count of clusters: 10
19.64075663591531 19.417472515521055
Count of clusters: 11
20.127588746514583 19.84128407389954
